## Problem 1: Difference fanout

>Given a list of numbers, for each number generate a list of the differences between it and $n_{fanout}$ (known as the **fanout** value) following numbers in the list. Return a list of all the lists generated for each number. For members in the list that have fewer than $n_{fanout}$ following members, calculate as many differences as possible. For example, suppose we want to compute the difference fanout on the list `[3, 2, 4, 6, 1]` with a fanout value of 3. Then we would compute:
 - $3 \rightarrow [2 - 3, 4 - 3, 6 - 3]$
 - $2 \rightarrow [4 - 2, 6 - 2, 1 - 2]$
 - $4 \rightarrow [6 - 4, 1 - 4]$
 - $6 \rightarrow [1 - 6]$
 - $1 \rightarrow []$
 
``` Python
# example behavior
>>> difference_fanout([3, 2, 4, 6, 1], 3)
[[-1, 1, 3], [2, 4, -1], [2, -3], [-5], []]
```

You will want to know about [lists](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Basic_Objects.html#Lists), [indexing & slicing](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html#Introducing-Indexing-and-Slicing) lists, and [for-loops](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/ForLoops.html) to solve this problem.

For extra credits (and some extra fun!), try to write your function only using [list comprehension](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#List-&-Tuple-Comprehensions). 

### Solution: difference_fanout using for-loops
We will naturally tackle this problem by performing nested for-loops. The outermost for-loop will loop over each number in the list. We will refer to this number as the "base number". We will want the inner for-loop to iterate ahead of the base number so that we can compute the differences between it and its $n_{fanout}$ neighbors. We will need to take care re-initialize our intermediate list of differences for each new base number, otherwise each subtraction will get appended to one long list. 

```python
def difference_fanout(l, fanout):
    """ Return a list of differences for 
        each value with its following terms
        
        Parameters
        ----------
        l: List[Number]
            Input list of base numbers.
            
        fanout: int
            Number of neighbors to compute differences against.
        
        Returns
        -------
        List[List[Number]]
    """
    all_fanouts = []  # will store each list of fanouts
    for i, base_number in enumerate(l):
        # `base_fanout` will store the differences between 
        # the base number's successive neighbors and base number
        base_fanout = []  
        for neighbor in l[i+1: i+1+fanout]:
            base_fanout.append(neighbor - base_number)
            
        all_fanouts.append(base_fanout)
    return all_fanouts
```

Note our use of [enumerate](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Iterables.html#Enumerating-iterables); this permits us to simultaneously access our base number, which we use in the subtraction, as well as its index-position within the list `l`, which we use to determine the neighbors. 

Next, you may be concerned that our inner-loop will attempt to iterate beyond the end of the list. Consider the case in which `base_number` is the final element in `l`, thus `l[i+1: i+1+fanout]` would be equivalent to `l[len(l): len(l)+fanout]` - the stopping point for this slice clearly reaches beyond the extent of `l` (assuming `fanout > 0`). Fortunately, this is not an oversight on our part. While indexing a list outside of its bounds will raise an error, recall that [a slice will automatically limit itself to be within bounds of a given sequence](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html#Handling-out-of-bounds-indices). That is, `l[i+1: i+1+fanout]` actually behaves like `l[min(i, len(l)-1): min(len(l), i+1+fanout)]` (assuming we are dealing only with positive indices and non-empty lists). Thus our inner-loop will naturally limit itself. In the case that `base_number` is the final element in `l`, the inner-loop will exit immediately, leaving `base_fanout` empty. Although somewhat obscure, this is an important aspect of Python's slicing mechanism to keep in mind.

### Solution: difference_fanout using list comprehensions
We can make judicious use of nested [list comprehensions](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#List-&-Tuple-Comprehensions) to simplify our solution. Although the syntax may appear to be convoluted at first glance, it permits us proceed without worrying about initializing multiple empty lists and appending to them at the right points in our nested for-loops

``` Python
def difference_fanout(l, fanout):
    """ Return a list of differences for 
        each value with its following terms
        
        Parameters
        ----------
        l: List[Number]
            Input list
            
        fanout: int
            Number of neighbors to compute difference with
        
        Returns
        -------
        List[Number]
    """
    return [[neighbor - base for neighbor in l[i+1:i+1+fanout]] 
            for i,base in enumerate(l)]
```

See that the outermost list comprehension loops over the base number, as did the outer for-loop in the prior solution, and that the innermost list comprehension plays the same roll as the inner for-loop.

There are fewer potential points of failure in this solution, as its conciseness removes the "moving parts" that had to be managed in the previous solution. This should help demonstrate the power of the comprehension expression syntax.  

### Extension
Recall from [earlier](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Functions.html#Functions-are-Objects) that functions are, under the hood, just objects with some special operations that allow you to "call" a function. This means that you can pass other functions as parameters into a function. It is especially powerful, since it enables us to generalize the purposes of our functions. For example, we don't have to limit our function to just computing the **difference** between members and their following terms; we can apply **any** *binary operation*. Instead of finding the difference, we can calculate the sum or product or even concatenate two strings for a list of string. The possibilities are limitless. 

Armed with this knowledge, we can generalize the code.
```Python
def apply_fanout(l, fanout, op):
    """ Return a list of outputs for each value 
        after applying a binary operation between 
        the value and its following terms
        
        Parameters
        ----------
        l: List[Any]
            Input list
        
        fanout: int
            Number of neighbors to apply the operation with
        
        op: Callable[[Any, Any], Any]
            Any binary operation to be applied to fanout-pairs
            of elements in `l`.
        
        Returns
        -------
        List[List[Any]]
    """
    return [[op(neighbor, base) for neighbor in l[i+1:i+1+fanout]] 
            for i,base in enumerate(l)]
```
Now, we can rewrite `difference_fanout` simply as
``` Python
def subtract(a, b): 
    return a - b

def difference_fanout(l, fanout):
    return apply_fanout(l, fanout, subtract)
```
We can easily change `subtract` to some other function for a totally different use. 

## Problem 2: within_margin_percentage
> An algorithm is required to test out what percentage of the parts that a factory is producing fall within a safety margin of the design specifications. Given a list of values recording the metrics of the manufactured parts, a list of values representing the desired metrics required by the design, and a margin of error allowed by the design, compute what fraction of the values are within the safety margin (`<=`)

``` Python
# example behavior
>>> within_margin_percentage(desired=[10.0, 5.0, 8.0, 3.0, 2.0],
...                          actual= [10.3, 5.2, 8.4, 3.0, 1.2],
...                          margin=0.5)
0.8
```

See that $4/5$ of the values fall within the margin of error: $1.2$ deviates from $2$ by more than $0.5$. 

Complete the following function; consider the edge case where `desired` and `actual` are empty lists.

```python
def within_margin_percentage(desired, actual, margin):
    """ Compute the percentage of values that fall within
        a margin of error of the desired values
        
        Parameters
        ----------
        desired: List[float]
            The desired metrics
        
        actual: List[float]
            The corresponding actual metrics. 
            Assume `len(actual) == len(desired)`
        
        margin: float
            The allowed margin of error
        
        Returns
        -------
        float
            The fraction of values where |actual - desired| <= margin
    """
    # YOUR CODE HERE
    pass
```

You will want to be familiar with [comparison operators](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/ConditionalStatements.html#Comparison-Operations), [control flow](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Introduction.html), and [indexing lists](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html#Introducing-Indexing-and-Slicing) lists to solve this problem.

### Solution: within_margin_percentage
This problem can solved by simply looping over the pairs of actual and desired values and tallying the pairs that fall within the margin:
``` Python
def within_margin_percentage(desired, actual, margin):
    """ Compute the percentage of values that fall within
        a margin of error of the desired values
        
        Parameters
        ----------
        desired: List[float]
            The desired metrics
        
        actual: List[float]
            The actual metrics
        
        margin: float
            The allowed margin of error
        
        Returns
        -------
        float
            The fraction of values where |actual - desired| <= margin
    """
    count = 0  # tally of how values are within margin
    total = len(desired)
    for i in range(total):
        if abs(desired[i] - actual[i]) <= margin:
            count += 1  # Equivalent to `count = count + 1`
    return count / total if total > 0 else 1.0
```

See that we handle the edge case where `desired` and `actual` are empty lists: the [inline if-else statement](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/ConditionalStatements.html#Inline-if-else-statements) `count / total if total > 0 else 1` will return `1` when `total` is 0: 
```python
>>> within_margin_percentage([], [], margin=0.5)
1.0
```
which is arguably the appropriate behavior for this scenario (no values fall outside of the margin). Had we not anticipated this edge case, `within_margin_percentage([], [], margin=0.5)` would raise `ZeroDivisionError`.

It is also possible to write this solution using the built-in `sum` function and a [generator comprehension](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#Creating-your-own-generator:-generator-comprehensions) that filters out those pairs of items that fall outside of the desired margin:

```python
def within_margin_percentage(desired, actual, margin):
    total = len(desired)
    count = sum(1 for i in range(total) if abs(actual[i] - desired[i]) <= margin)
    return  count / total if total > 0 else 1.0
```

It is debatable whether this refactored solution is superior to the original one - it depends largely on how comfortable you, and anyone else who will be reading your code, are with the generator comprehension syntax.

## Problem 3: is_palindrome
> A palindrome is a string that reads the same from left to right and from right to left. Strings like `racecar` and `Live on time, emit no evil` are palindromes. Notice that only valid alphanumeric characters are accounted for and that palindromes are not case-sensitive. Given a string, return whether or not it is a palindrome. 

```python
# example behavior
>>> is_palindrome("Step on no pets!")
True
>>> is_palindrome("'Tis not a palindrome")
False
>>> is_palindrome("Hi, I am Mai Ih")
True
```

### Tips 
[str.isalnum](https://docs.python.org/3/library/stdtypes.html#str.isalnum) returns whether or not a string has purely alphanumeric characters (it works for single-character strings too).
```python
>>> "I love Python".isalnum()
False
>>> "IlovePython".isalnum()
True
```

Consider using this along with `str.lower` to filter out ignored characters and to normalize all of the character casing in the string before assessing whether or not it is a palindrome. 


### Solution: is_palindrome
The simplest solution to this problem is the following, where we make use of the `str.join` function as well as slicing with a negative step:

```python
def is_palindrome(input_str):
    """ Given a string, determine if it is a palindrome.
        Whitespaces, character-casing, and non-alphanumeric  
        characters are all ignored.
        
        Parameters
        ----------
        s: str
            Input string
        
        Returns
        -------
        bool
    """
    filtered_str = "".join(c.lower() for c in input_str if c.isalnum())
    return filtered_str == filtered_str[::-1]
```

See that `(c.lower() for c in input_str if c.isalnum())` has the form of a [filtering generator comprehension](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Generators_and_Comprehensions.html#Creating-your-own-generator:-generator-comprehensions). Thus, 

```python
"".join(c.lower() for c in input_str if c.isalnum())
```
is equivalent to the long-form code:

```python
filtered_str = ""
for char in input_str:
    if char.isalnum():
        filtered_str += char.lower()
```
The generator comprehension expression is not only more concise and readable, but its use of `str.join` also makes it a more efficient means for constructing a new list. Each call to `filtered_str += c.lower()` in the long-form code creates a new string in memory, whereas `str.join` forms a single string as it consumes the input iterable.

Next, [recall that](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html#Slicing) `seq[::-1]` slices a sequence with a step of -1, which produces the sequence in *reverse order*. Thus `filtered_str == filtered_str[::-1]` allows us to compare the first character in `filtered_str` with the last and so on. This is equivalent to:

```python
is_equal = True
for i in range(len(filtered_str)//2): # recall:  5//2 -> 2, 6//2 -> 3
    if filtered_str[i] != filtered_str[-(i+1)]:
        is_equal = False
        break
```

The only downside to using slicing to perform this comparison is that it requires that a copy of `filtered_str` be created, whereas using the explicit for-loop does not. 

We must note that the performance differences pointed out here should only concern us if `is_palindrome` is potentially a performance bottleneck for our code. Although we want the reader to develop an intuition for writing efficient Python code, we discourage mangling code for the sake of premature optimization.

## Problem 4: concat_to_str
Sometimes it is very important to handle different input object types differently in a function. This problem will exercise your understanding of types, control-flow, dictionaries, and more.

>We want to encode a sequence of Python objects as a single string. The following describes the encoding method that we want to use for each type of object. Each object's transcription in should be separated by `" | "`, and the result should be one large string. 
- If the object is an integer, convert it into a string by spelling out each digit in base-10 in this format:
`142` $\rightarrow$ `one-four-two`; `-12` $\rightarrow$ `neg-one-two`. 
- If the object is a float, just append its integer part (obtained by rounding down) the same way and the string `"and float"`:
`12.324` $\rightarrow$ `one-two and float`. 
- If the object is a string, keep it as is.
- If the object is of any other type, return `'<OTHER>'`.
 

``` Python
# example behavior
>>> s = concat_to_str([12,-14.23,"hello", True,
...                    "Aha", 10.1, None, 5])
>>> print(s)
one-two | neg-one-four and float | hello | <OTHER> | Aha | one-zero and float | <OTHER> | five
```

**Tips**: check out the `isinstance` function introduced [here](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Basic_Objects.html) for handling different types. Also, consider creating a helper function for the conversion from integer to our special-format string, since we have to do it twice. It's always good to extrapolate repeated tasks into functions. You'll also need to hard-code the conversion from each digit to its English spell-out. 

### Solution: concat_to_str
Our solution is broken down into three simple functions. `int_to_str` is used to map signed integers to English words. `item_to_transcript` is capable of mapping an object of any type to its string representation, in accordance with the prescription made by the problem statement. Finally, `concat_to_str` orchestrates these two helper functions, looping over each object in our input list, mapping each object to its string representation, and joining these strings with `' | '`. 

```python
def int_to_str(n):
    """ 
    Takes an integer and formats it into a special string 
        e.g. 142 -> "one-four-two"
             -12 -> "neg-one-two"
    """
    mapping = {"0": "zero", "1": "one", "2": "two", "3": "three",
               "4": "four", "5": "five", "6": "six", "7": "seven",
               "8": "eight", "9": "nine", "-": "neg"}
    return "-".join(mapping[digit] for digit in str(n))
    
def item_to_transcript(item):
    """ Any -> str """
    if isinstance(item, bool): return '<OTHER>'
    if isinstance(item, int): return int_to_str(item)
    if isinstance(item, float): return int_to_str(int(item)) + " and float"
    if isinstance(item, str): return item
    return '<OTHER>'

def concat_to_str(l):
    """ 
    Maps a list of objects to their string 
    representations concatenated together.

    Parameters
    ----------
    l: List[Any]
        Input list of objects

    Returns
    -------
    str

    Examples
    --------
    >>> concat_to_str([1, None, 'hi', 2.0])
    one | <OTHER> | hi | two and float
    """
    return " | ".join(item_to_transcript(item) for item in l)
```

We use the `str.join` function along with a generator comprehensions in a couple places in our solution. Recall that 
```python
"<hi>".join(x for x in some_iterable_of_strings)
```
is equivalent to the long-form code:
```python
out = ""
for x in some_iterable_of_strings:
    out += "<hi>" + x
``` 

`int_to_str` plays a clever trick to convert each integer, digit-by-digit, into its string form - it calls `str` on the integer. This converts the integer into a string, which is a [sequence](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/SequenceTypes.html). This permits us to access each digit of the integer and even iterate over them:

```python
# casting an integer to a string makes its
# sign and digits accessible via indexing/iteration
>>> x = str(-123)
>>> x
'-123'
>>> x[0]
'-'
>>> x[-1]
'3'
```
Thus, in total `"-".join(mapping[digit] for digit in str(n))` is responsible for casting an integer to a string, iterating over each of its digits and mapping them to their corresponding word using the [dictionary](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/DataStructures_II_Dictionaries.html) that we defined in the function.

`item_to_transcript` it an especially slick function. First, let's make clear the fancy use of the inline syntax here. This function:
```python
def item_to_transcript(item):
    """ Any -> str """
    if isinstance(item, bool): return '<OTHER>'
    if isinstance(item, int): return int_to_str(item)
    if isinstance(item, float): return int_to_str(int(item)) + " and float"
    if isinstance(item, str): return item
    return '<OTHER>'
```
is entirely equivalent to this function:
```python
def item_to_transcript_alt(item):
    """ Any -> str """
    if isinstance(item, bool): 
        return '<OTHER>'
    elif isinstance(item, int): 
        return int_to_str(item)
    elif isinstance(item, float): 
        return int_to_str(int(item)) + " and float"
    elif isinstance(item, str): 
        return item
    else:
        return '<OTHER>'
```
The latter uses the familiar pattern of [if-elif-else](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/ConditionalStatements.html) statements and makes for a completely satisfactory version of the function. See, however, that each of the [multiple return statements](https://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Functions.html#Multiple-return-Statements) in `item_to_transcript` guarantees the same logic, in that if a condition is meant a value will be returned by the function and none of its subsequent code can be visited. That is, if `item` is an integer the second if-condition will evaluate to `True` and `int_to_str(item)` will be returned, immediately expelling the point of execution from the body of the function.

Ultimately, the preference of one function over the other is merely a matter of stylistic preference. You also have likely noted the peculiar in-line `if-return` expressions. These too are only stylistic choices; 
```python
if isinstance(item, int): return int_to_str(item)
```
is no different from
```python
if isinstance(item, int): 
    return int_to_str(item)
```
The use of in-line `if-return` expressions in `item_to_transcript` does a nice job emphasizing the dictionary-like mapping behavior of the function: the form of the code suits its functionality nicely. That being said, these should generally be used sparingly. Some may call this a "cute" trick. And it is. This code is cute. I write cute code.

Finally, you may have noticed what looks like a redundancy in our code: our first `if` statement returns `'<OTHER>'` if `item` is `True` or `False`, and our final line of code returns `'<OTHER>'` if none of the preceding conditions were met (i.e. `item` is not a `bool`, `int`, `float`, or `str` type object). Why then did we not just merge our first `if` clause with this ultimate catch-all? The reason is that `True` and `False` are not only instances of the boolean type, they are also integers! `True` behaves like the integer `1` and `False` like `0`:

```python
>>> isinstance(True, int) and isinstance(True, bool)
True

>>> 3*True + True - False
4
```

Thus, had we not taken care to check for booleans up front, `True` and `False` would have been mapped to `'one'` and `'zero'`, respectively, rather than `'<OTHER>'`. This is a relatively subtle edge case to catch.